In [1]:
from data.echr import EchrDataset
ds = EchrDataset(data_path="data/echr", pseudonymize=True, mode='scrubbed')

/opt/conda/envs/llm-pbe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


echr, EchrDataset
2023-12-07 18:57:35,052 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY

--------------------------------------------------------------------------------
anon_token   : <MASK>
anonymize    : True
ner          : flair
ner_model    : flair/ner-english-ontonotes-large
tag_n_batches: 10000
--------------------------------------------------------------------------------



Generating train split: 0 examples [00:00, ? examples/s]


--------------------------------------------------------------------------------
Length of data: 262580. Scrubbing from 0 to 118161 (Total=118161).
--------------------------------------------------------------------------------



Generating train split: 7 examples [00:00, 23.33 examples/s]


--------------------------------------------------------------------------------
<MASK>  On <MASK> <MASK> informed the applicant’s lawyer that the time-limit had been extended until such time as the applicant and his child were fully recovered.
--------------------------------------------------------------------------------



Generating train split: 2312 examples [00:45, 49.44 examples/s]

In [3]:
import json
import re

def preprocess(ds, pii_type='PERSON', min_prompt_length=200, min_target_length=10):
    json_data = []
    pii = ds.train_set()[pii_type]
    for i in range(len(ds.train_set())):
        entry = pii[i]
        if entry != "ListPII(data=[])":
            data = json.loads(str(entry))
            target = [entity['text'] for entity in data['data'] if entity['entity_class'] == pii_type]
            start_indices = [entity['start'] for entity in data['data'] if entity['entity_class'] == pii_type]
            data_zipped = list(zip(target, start_indices))
            sorted_data = sorted(data_zipped, key=lambda x: x[1])
            for entry in sorted_data:
                target, start_index = entry
                # Get the first entry with min prompt and target length
                if start_index >= min_prompt_length and len(target) >= min_target_length:
                    json_entry = {}
                    json_entry["target"] = target
                    json_entry["prompt"] = ds.train_set()["text"][i][:start_index]
                    json_entry["prompt"] = re.sub(r'^\d+\.\s+', '', json_entry['prompt']) # Remove the beginning digits in the prompt
                    json_data.append(json_entry)
                    break
    return json_data

json_data = preprocess(ds)

# Writing data to a JSONL file
with open('context.jsonl', 'w') as file:
    for entry in json_data:
        json.dump(entry, file)
        file.write('\n')  # Newline character to separate JSON objects
        

KeyError: "Column PERSON not in the dataset. Current columns in the dataset: ['text']"